# Week 2 — Notebook 1: Dataset Preparation & DPO Pair Construction

This notebook covers:
1. Loading and inspecting raw data (~90k rows)
2. De-identification with standard libraries (no LLMs)
3. Data quality assessment
4. Extracting existing chosen/rejected pairs **with their pre-computed scores**
5. Two filtering strategies using those real scores:
   - **Score delta threshold** (keep pairs where chosen >> rejected)
   - **Score floor + delta** (additionally require chosen to clear a minimum quality bar)
6. Sample efficiency analysis

---
> **GPU requirement:** None — CPU/data-engineering only.  
> **Estimated runtime:** ~10–20 min depending on dataset size.

## 0. Environment & Imports

In [ ]:
# Install dependencies if needed (comment out after first run)
# !pip install datasets pandas pyarrow presidio-analyzer presidio-anonymizer spacy tqdm
# !python -m spacy download en_core_web_lg

In [ ]:
import os
import json
import random
import warnings
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
random.seed(42)
np.random.seed(42)

# Paths
DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)

print("Imports OK")

## 1. Load Raw Dataset

We use the **Argilla Distilabel Customer Support** dataset as a realistic proxy.  
Replace `DATASET_PATH` with your own Spark-processed parquet/JSONL if you have it.

In [ ]:
# --- Option A: Load from HuggingFace Hub (public proxy dataset) ---
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
df = dataset.to_pandas()

# --- Option B: Load your own data ---
# df = pd.read_parquet(DATA_DIR / "rollouts_raw.parquet")
# df = pd.read_json(DATA_DIR / "rollouts_raw.jsonl", lines=True)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head(3)

In [ ]:
# Quick stats
print(df.dtypes)
print("\nNull counts:")
print(df.isnull().sum())

## 2. De-identification (Standard Libraries)

Using **Microsoft Presidio** — no LLM calls, fully deterministic.

In [ ]:
try:
    from presidio_analyzer import AnalyzerEngine
    from presidio_anonymizer import AnonymizerEngine

    analyzer = AnalyzerEngine()
    anonymizer = AnonymizerEngine()
    PRESIDIO_AVAILABLE = True
    print("Presidio loaded.")
except ImportError:
    PRESIDIO_AVAILABLE = False
    print("Presidio not installed — skipping de-id (install presidio-analyzer presidio-anonymizer spacy + en_core_web_lg)")


def deidentify(text: str) -> str:
    """Replace PII entities with type placeholders, e.g. <PERSON>."""
    if not PRESIDIO_AVAILABLE or not isinstance(text, str):
        return text
    results = analyzer.analyze(text=text, language="en")
    anonymized = anonymizer.anonymize(text=text, analyzer_results=results)
    return anonymized.text


# Test
sample = "Hi, I'm John Smith and my email is john@example.com, phone 555-1234."
print("Before:", sample)
print("After: ", deidentify(sample))

In [ ]:
# Apply to the prompt column (adjust column name to your schema)
PROMPT_COL = "prompt"  # change if your column is named differently

if PROMPT_COL in df.columns and PRESIDIO_AVAILABLE:
    tqdm.pandas(desc="De-identifying prompts")
    df["prompt_clean"] = df[PROMPT_COL].progress_apply(deidentify)
    changed = (df["prompt_clean"] != df[PROMPT_COL]).sum()
    print(f"Records modified by de-id: {changed}/{len(df)} ({changed/len(df)*100:.1f}%)")
else:
    df["prompt_clean"] = df.get(PROMPT_COL, "")

## 3. Data Quality Assessment

In [ ]:
def quality_metrics(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """Compute basic quality signals on a text column."""
    d = df.copy()
    d["len_chars"] = d[text_col].str.len()
    d["len_words"] = d[text_col].str.split().str.len()
    d["is_empty"] = d[text_col].str.strip().str.len() == 0
    d["has_url"] = d[text_col].str.contains(r"https?://", na=False)
    d["is_duplicate"] = d.duplicated(subset=[text_col], keep=False)
    return d


df = quality_metrics(df, "prompt_clean")

print("=== Quality Summary ===")
print(f"Total rows        : {len(df):,}")
print(f"Empty prompts     : {df['is_empty'].sum():,}")
print(f"Duplicate prompts : {df['is_duplicate'].sum():,}")
print(f"Contains URL      : {df['has_url'].sum():,}")
print(f"\nPrompt length (words):")
print(df["len_words"].describe().round(1).to_string())

In [ ]:
# Filter out bad rows
before = len(df)
df = df[
    ~df["is_empty"]
    & ~df.duplicated(subset=["prompt_clean"], keep="first")
    & df["len_words"].between(5, 2000)
].reset_index(drop=True)
print(f"Rows after quality filter: {len(df):,} (removed {before - len(df):,})")

## 4. Extract Chosen / Rejected Text and Their Pre-Computed Scores

The dataset already ships with `score_chosen` and `score_rejected` — no VJ needed here.

In [ ]:
def extract_last_assistant(messages) -> str:
    """Pull the final assistant turn from a list-of-dicts conversation."""
    if isinstance(messages, list):
        for msg in reversed(messages):
            if isinstance(msg, dict) and msg.get("role") == "assistant":
                return msg.get("content", "")
    return str(messages)


# Extract text
df["response_chosen"]   = df["chosen"].apply(extract_last_assistant)
df["response_rejected"] = df["rejected"].apply(extract_last_assistant)

# Scores are already floats in the dataset
# UltraFeedback uses a 1–5 scale; normalise to [0, 1]
SCORE_MAX = 5.0
df["score_chosen"]   = pd.to_numeric(df["score_chosen"],   errors="coerce") / SCORE_MAX
df["score_rejected"] = pd.to_numeric(df["score_rejected"], errors="coerce") / SCORE_MAX

# Drop rows where scores are missing
before = len(df)
df = df.dropna(subset=["score_chosen", "score_rejected"]).reset_index(drop=True)
print(f"Rows with valid scores: {len(df):,} (dropped {before - len(df):,} missing-score rows)")

print("\nScore distributions (normalised 0–1):")
print(df[["score_chosen", "score_rejected"]].describe().round(3))

df["score_delta"] = df["score_chosen"] - df["score_rejected"]
print(f"\nscore_delta stats:\n{df['score_delta'].describe().round(3)}")
print(f"Pairs where chosen > rejected: {(df['score_delta'] > 0).mean()*100:.1f}%")

## 5. Strategy A — Score Delta Threshold

Keep pairs where `score_chosen - score_rejected ≥ DELTA_THRESHOLD`.  
A tight threshold gives cleaner signal but fewer pairs (see §7 for the funnel).

In [ ]:
DELTA_THRESHOLD = 0.10   # ~0.5 points on the original 1–5 scale

df_delta = df[df["score_delta"] >= DELTA_THRESHOLD].copy()

print(f"Pairs surviving delta >= {DELTA_THRESHOLD} filter:")
print(f"  {len(df_delta):,} / {len(df):,} ({len(df_delta)/len(df)*100:.1f}%)")
print(df_delta["score_delta"].describe().round(3))

## 6. Strategy B — Score Floor + Delta

Also require the chosen response to clear a **minimum quality floor**.  
This prevents pairs where both responses are low-quality but one is marginally better.

In [ ]:
CHOSEN_FLOOR  = 0.70   # chosen must score >= 3.5/5 normalised
DELTA_MIN     = 0.15   # beat rejected by >= 0.75 pts on original scale
MAX_PAIRS     = 1000   # cap for training efficiency; set None to keep all

df_floor = df[
    (df["score_chosen"] >= CHOSEN_FLOOR)
    & (df["score_delta"] >= DELTA_MIN)
].copy()

print(f"Pairs surviving floor + delta filter: {len(df_floor):,} / {len(df):,} ({len(df_floor)/len(df)*100:.1f}%)")

# Cap to MAX_PAIRS, sampling randomly to avoid positional bias
if MAX_PAIRS and len(df_floor) > MAX_PAIRS:
    df_floor = df_floor.sample(MAX_PAIRS, random_state=42).reset_index(drop=True)
    print(f"Sampled down to {MAX_PAIRS} pairs (random_state=42)")

print(f"\nFinal pairs     : {len(df_floor):,}")
print(f"Chosen avg score : {df_floor['score_chosen'].mean():.3f}")
print(f"Rejected avg score: {df_floor['score_rejected'].mean():.3f}")
print(f"Mean delta        : {df_floor['score_delta'].mean():.3f}")

## 7. Sample Efficiency Summary

In [ ]:
funnel = pd.DataFrame([
    {"stage": "Raw data",                           "rows": len(dataset)},
    {"stage": "After quality filter",               "rows": len(df)},
    {"stage": "Strategy A (delta >= 0.10)",         "rows": len(df_delta)},
    {"stage": "Strategy B (floor + delta)",         "rows": len(df[
        (df["score_chosen"] >= CHOSEN_FLOOR) & (df["score_delta"] >= DELTA_MIN)
    ])},
    {"stage": f"Strategy B capped @ {MAX_PAIRS}",   "rows": len(df_floor)},
])
funnel["retention_%"] = (funnel["rows"] / funnel["rows"].iloc[0] * 100).round(1)
print(funnel.to_string(index=False))

## 8. Export DPO-Ready Datasets

In [ ]:
def to_dpo_format(df: pd.DataFrame) -> pd.DataFrame:
    """Convert to TRL DPO trainer schema, keeping score_chosen for SFT filtering in NB02."""
    return pd.DataFrame({
        "prompt":        df["prompt_clean"],
        "chosen":        df["response_chosen"],
        "rejected":      df["response_rejected"],
        "score_chosen":  df["score_chosen"],
        "score_rejected":df["score_rejected"],
        "score_delta":   df["score_delta"],
    })


for name, subset in [("delta", df_delta), ("floor", df_floor)]:
    out = to_dpo_format(subset)
    path = DATA_DIR / f"dpo_{name}.jsonl"
    out.to_json(path, orient="records", lines=True)
    print(f"Saved {len(out):,} pairs → {path}")

# Also save as HuggingFace Dataset (used by NB02)
ds_floor = Dataset.from_pandas(to_dpo_format(df_floor))
ds_floor.save_to_disk(str(DATA_DIR / "dpo_floor_hf"))
print("HuggingFace dataset saved.")

---
## Summary

| Dataset | Pairs | Filter |
|---------|-------|--------|
| `dpo_delta.jsonl` | ~N | score_delta ≥ 0.10 |
| `dpo_floor.jsonl` | ~N | score_chosen ≥ 0.70 AND delta ≥ 0.10 |

Both files follow TRL DPO trainer schema: `{prompt, chosen, rejected}`.

**Next:** `02_finetuning_qlora.ipynb` — SFT warm-up on high-score chosen responses → DPO.